In [32]:
from config import username, password, hostname, port
from pprint import pprint
from sqlalchemy import create_engine
import json
import pandas as pd

In [6]:
with open("drugs.json") as json_file:
    drugs = json.load(json_file)
    
pprint(drugs)

[{'alternative_identifiers': {'BindingDB': '13530',
                              'ChEBI': '45783',
                              'ChEMBL': 'CHEMBL941',
                              'ChemSpider': '5101',
                              'Drugs.com': 'Drugs.com Drug Page',
                              'Human Metabolome Database': 'HMDB0014757',
                              'KEGG Drug': 'D01441',
                              'PDBe Ligand': 'STI',
                              'PharmGKB': 'PA10804',
                              'PubChem Compound': '5291',
                              'PubChem Substance': '46505055',
                              'RxList': 'RxList Drug Page',
                              'RxNav': '282388',
                              'Therapeutic Targets Database': 'DNC001383',
                              'Wikipedia': 'Imatinib',
                              'ZINC': 'ZINC000019632618'},
  'drug_bank_id': 'DB00619',
  'smiles': 'CN1CCN(CC2=CC=C(C=C2)C(=O)NC2=CC(NC3

In [44]:
# Initialize data frames to go with SQL Tables
action_df = pd.DataFrame(columns=['target_id', 'action'])
alt_id_df = pd.DataFrame(columns=['drug_id', 'location', 'name'])
drug_df = pd.DataFrame(columns=['drug_id', 'smiles'])
target_df = pd.DataFrame(columns=['drug_id', 'gene_name'])

print(action_df.head())
print(alt_id_df.head())
print(drug_df.head())
print(target_df.head())

Empty DataFrame
Columns: [target_id, action]
Index: []
Empty DataFrame
Columns: [drug_id, location, name]
Index: []
Empty DataFrame
Columns: [drug_id, smiles]
Index: []
Empty DataFrame
Columns: [drug_id, gene_name]
Index: []


In [43]:
# This is a test script and should not be used for deployment
#drug = drugs[0]

#alt_ids = drug['alternative_identifiers']
#targets = drug['targets']

#drug_df = drug_df.append({
#    'drug_id': drug['drug_bank_id'], 
#    'smiles': drug['smiles']
#}, ignore_index=True)

#for key, value in alt_ids.items():
#    alt_id_df = alt_id_df.append({
#        'drug_id': drug['drug_bank_id'], 
#        'location': key, 
#        'name': value
#    }, ignore_index=True)
    
#for target in targets:
#    target_df = target_df.append({
#        'drug_id': drug['drug_bank_id'], 
#        'gene_name': target['gene_name']
#    }, ignore_index=True)
#    print(f"Target appended at index {target_df.last_valid_index()}")
    
#    for action in target['actions']:
#        action_df = action_df.append({
#            'target_id': target_df.last_valid_index(), 
#            'action': action
#        }, ignore_index=True)
        
#print(action_df.head())
#print(alt_id_df.head())
#print(drug_df.head())
#print(target_df.head())


In [45]:
for drug in drugs:
    
    drug_df = drug_df.append({
        'drug_id': drug['drug_bank_id'], 
        'smiles': drug['smiles']
    }, ignore_index=True)

    try:
        alt_ids = drug['alternative_identifiers']
        for key, value in alt_ids.items():
            alt_id_df = alt_id_df.append({
                'drug_id': drug['drug_bank_id'], 
                'location': key, 
                'name': value
            }, ignore_index=True)
    except KeyError:
        print(f"{drug['drug_bank_id']}: alt identifiers not found")

    try:
        targets = drug['targets']
        for target in targets:
            target_df = target_df.append({
                'drug_id': drug['drug_bank_id'], 
                'gene_name': target['gene_name']
            }, ignore_index=True)
            
            try:
                for action in target['actions']:
                    action_df = action_df.append({
                        'target_id': target_df.last_valid_index(), 
                        'action': action
                    }, ignore_index=True)
            except KeyError:
                print(f"{drug['drug_bank_id']}: actions not found")
                
    except KeyError:
        print(f"{drug['drug_bank_id']}: targets not found")

DB14093: targets not found
DB00173: actions not found
DB00173: actions not found
DB00173: actions not found
DB00173: actions not found
DB00173: actions not found
DB00173: actions not found
DB00173: actions not found
DB00173: actions not found
DB00173: actions not found
DB00173: actions not found
DB00173: actions not found
DB05196: targets not found


In [46]:
action_df.head()

,target_id,action
0,0,Inhibitor
1,1,Antagonist
2,1,Multitarget
3,2,Inhibitor
4,3,Antagonist


In [47]:
alt_id_df.head()

,drug_id,location,name
0,DB00619,Human Metabolome Database,HMDB0014757
1,DB00619,KEGG Drug,D01441
2,DB00619,PubChem Compound,5291
3,DB00619,PubChem Substance,46505055
4,DB00619,ChemSpider,5101


In [48]:
drug_df.head()

,drug_id,smiles
0,DB00619,CN1CCN(CC2=CC=C(C=C2)C(=O)NC2=CC(NC3=NC=CC(=N3...
1,DB01048,NC1=NC2=C(N=CN2[C@@H]2C[C@H](CO)C=C2)C(NC2CC2)=N1
2,DB14093,[3H]C1C([3H])C2=CC(=O)C([3H])C([3H])[C@]2(C)[C...
3,DB00173,NC1=C2NC=NC2=NC=N1
4,DB00734,CC1=C(CCN2CCC(CC2)C2=NOC3=C2C=CC(F)=C3)C(=O)N2...


In [49]:
target_df.head()

,drug_id,gene_name
0,DB00619,BCR
1,DB00619,KIT
2,DB00619,RET
3,DB00619,NTRK1
4,DB00619,CSF1R


In [41]:
engine = create_engine(f"postgres://{username}:{password}@{hostname}:{port}/drugbank_db")

In [42]:
drug_df.to_sql('drug', con=engine, if_exists='append', index=False)
alt_id_df.to_sql('alt_identifier', con=engine, if_exists='append', index_label='alt_identifier_id')
target_df.to_sql('target', con=engine, if_exists='append', index_label='target_id')
action_df.to_sql('action', con=engine, if_exists='append', index_label='action_id')